In [3]:
import open3d as o3d
import numpy as np
import sys

In [5]:
sys.path.append('../sample')
from readdatas import *

In [6]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [7]:
o3d.visualization.draw_geometries([pcd.pcd])

In [5]:
csf = CsfGroundExtractor()
csf.set_data(pcd)
csf._process_data()

In [15]:
ran = RansacGroundExtractor()
ran.set_data(pcd)
ran._process_data()

In [8]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [9]:
ground= gpf.np_to_o3d(pcd.ground)

In [10]:
o3d.visualization.draw_geometries([ground])

In [14]:
reg = ReGrowSegment()
reg.set_data(pcd)
reg._process_data()

[Open3D INFO] Picked point #18116 (-5.2, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #28121 (-1.8, -12., -1.6) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [15]:
no_ground = gpf.np_to_o3d(pcd.no_ground)
no_paves = ground.select_by_index(pcd.paves,invert = True)

In [16]:
paves = ground.select_by_index(pcd.paves)
o3d.visualization.draw_geometries([paves])

In [25]:
paves.paint_uniform_color([1,0,0])

PointCloud with 60751 points.

In [26]:
o3d.visualization.draw_geometries([no_paves,paves,no_ground])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 


In [7]:
dri = DriPathSegment()
dri.set_data(pcd)
dri._process_data()

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D INFO] Picked point #15094 (-14., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #17843 (-5.4, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #47538 (1.8, -10., -1.5) to add in queue.
[Open3D INFO] Picked point #56860 (4.8, -5.8, -1.5) to add in queue.
[Open3D INFO] Picked point #67809 (5.3, -2.0, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #69018 (5.3, 5.9, -1.5) to add in queue.


In [8]:
paves  = ground.select_by_index(pcd.no_paves, invert = True)

In [9]:
o3d.visualization.draw_geometries([paves])

In [15]:
def euclidean_cluster(dps,cloud, tolerance=0.2):
    seed = dps.seed_select(cloud)
    no_paves = dps.find_no_paves(cloud)
    paves = np.asarray(cloud.points)
    no_paves = dps.find_index(paves, no_paves)
    kdtree = o3d.geometry.KDTreeFlann(cloud)
    paves = np.array(seed)
    while len(seed) > 0 :
        seed_now = seed[0]
        k, idx, _ = kdtree.search_radius_vector_3d(cloud.points[seed_now], tolerance)
        if k == 1 :
            continue
        idx = np.array(idx)
        idx = idx[np.isin(idx, paves, invert=True)]
        idx = idx[np.isin(idx, no_paves, invert=True)]
        paves = np.append(paves, idx)
        seed = np.append(seed,idx)
        seed = np.delete(seed,[0])

    return paves

In [21]:
pave = euclidean_cluster(dri,paves, tolerance=0.15)

[Open3D INFO] Picked point #15063 (-7.0, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #37270 (-0.21, -11., -1.5) to add in queue.
[Open3D INFO] Picked point #56882 (4.6, -9.3, -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [20]:
ground1 = paves.select_by_index(pave)
o3d.visualization.draw_geometries([ground1])